In [10]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
from transformers.models.bert.configuration_bert import BertConfig


In [26]:
def dnabert2(sequence, tokenizer, model, pooling):

  # with torch.no_grad(): 
  inputs = tokenizer(sequence, return_tensors = 'pt')["input_ids"]
  hidden_states = model(inputs)[0] # [1, sequence_length, 768]

  # embedding with mean pooling
  if pooling == 'mean':
    embedding = torch.mean(hidden_states[0], dim=0)

  # embedding with max pooling
  else:
    embedding = torch.max(hidden_states[0], dim=0)[0]

  return embedding.detach().numpy()


In [34]:
# Função para ler o CSV e gerar os encodings
def process_csv(train_file, param):
  # Lê o CSV ignorando a primeira linha
  df = pd.read_csv(train_file)    

  # Fetch pooling marameter
  pooling = param[0]
    
  tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
  #model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
  
  config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
  model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)

  # Aplica o encoding do dnabert2 nas sequencias 
  encoded_sequences = [dnabert2(seq, tokenizer, model, pooling) for seq in df['sequence']]
       
  labels = df['label']  
  
  return encoded_sequences, labels


In [35]:
csv_file = 'encode.csv'
encoded_sequences, labels = process_csv(csv_file, ['mean'])

/home/stavisa/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
encoded_sequences2, labels = process_csv(csv_file, ['max'])

/home/stavisa/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
